# **Project 3- Team 7**
## By: Juan Marin, James Lamotte, Zack Crowley, Matusola Bein 


### Import dependencies

In [39]:
# Dependencies
import pandas as pd
import geopandas as gpd
import shapely.geometry
import os
import numpy as np
import json
import datetime
import pymongo
from pymongo import MongoClient, GEOSPHERE
from pymongo.errors import (PyMongoError, BulkWriteError)
import argparse, urllib
from bson.json_util import dumps, loads 
from collections import defaultdict
from typing import Any
from bson import ObjectId

## Import .csv Data

- First read in the raw .csv data from the Zillow Home Value Index ("zhvi_allhomes_ts_county.csv") and Nat'l Risk Index data ("natl_risk_index_counties.csv") into DataFrames

In [9]:
# Zillow Home Value Index County Time Series data:
homes_file = "resources/zhvi_allhomes_ts_county.csv"
raw_homes_df = pd.read_csv(homes_file)
raw_homes_df.head()

,RegionID,SizeRank,RegionName,RegionType,StateName,State,Metro,StateCodeFIPS,MunicipalCodeFIPS,2000-01-31,...,2022-03-31,2022-04-30,2022-05-31,2022-06-30,2022-07-31,2022-08-31,2022-09-30,2022-10-31,2022-11-30,2022-12-31
0,3101,0,Los Angeles County,county,CA,CA,"Los Angeles-Long Beach-Anaheim, CA",6,37,217219.0,...,849467.0,867274.0,882623.0,884311.0,883805.0,870575.0,859871.0,848269.0,846262.0,845596.0
1,139,1,Cook County,county,IL,IL,"Chicago-Naperville-Elgin, IL-IN-WI",17,31,173902.0,...,301166.0,303308.0,306745.0,309530.0,311511.0,311997.0,312558.0,313027.0,313775.0,312317.0
2,1090,2,Harris County,county,TX,TX,"Houston-The Woodlands-Sugar Land, TX",48,201,115286.0,...,269990.0,275086.0,280076.0,283929.0,286477.0,288032.0,288399.0,289143.0,289720.0,289591.0
3,2402,3,Maricopa County,county,AZ,AZ,"Phoenix-Mesa-Chandler, AZ",4,13,142739.0,...,457069.0,468583.0,477443.0,483784.0,481374.0,474787.0,464870.0,459841.0,454753.0,451699.0
4,2841,4,San Diego County,county,CA,CA,"San Diego-Chula Vista-Carlsbad, CA",6,73,225274.0,...,887685.0,906908.0,920069.0,919535.0,908231.0,897158.0,890169.0,885101.0,880975.0,874095.0


In [10]:
# Nat'l Risk Index County level data:
risk_file = "resources/natl_risk_index_counties.csv"
raw_risk_df = pd.read_csv(risk_file)
raw_risk_df.head()

,OBJECTID,NRI_ID,STATE,STATEABBRV,STATEFIPS,COUNTY,COUNTYTYPE,COUNTYFIPS,STCOFIPS,POPULATION,...,WNTW_EALA,WNTW_EALT,WNTW_EALS,WNTW_EALR,WNTW_RISKS,WNTW_RISKR,NRI_VER,GlobalID,SHAPE_Length,SHAPE_Area
0,1,C21115,Kentucky,KY,21,Johnson,County,115,21115,23356,...,4.235939,47363.199731,19.448529,Relatively Moderate,14.131237,Relatively Low,November 2021,{55EC05F7-E576-469E-B728-EF2D3AEFD6DA},1.553484,0.070028
1,2,C21117,Kentucky,KY,21,Kenton,County,117,21117,159720,...,44.606252,64259.532691,21.530408,Relatively Moderate,12.470040,Relatively Low,November 2021,{0868CDAC-FE0A-489E-8E2E-D5B7EB7AD660},1.094178,0.044207
2,3,C21119,Kentucky,KY,21,Knott,County,119,21119,16346,...,0.023091,30809.754620,16.851393,Relatively Low,14.466270,Relatively Low,November 2021,{A448B94E-9172-4B0C-B74B-321B3CF39E66},1.694376,0.092941
3,4,C21121,Kentucky,KY,21,Knox,County,121,21121,31883,...,0.082573,61427.308851,21.209328,Relatively Moderate,19.585915,Relatively Moderate,November 2021,{BA4C9D7D-5891-4EEE-9B3B-BF1F9BA5AA4A},1.920543,0.101520
4,5,C21123,Kentucky,KY,21,Larue,County,123,21123,14193,...,246.668438,12870.385216,12.597091,Relatively Low,7.715952,Very Low,November 2021,{55A74C8C-E1FD-4C88-8343-D977E94CD0B1},1.830220,0.069616


### Clean raw .csv date:
- First, the Zillow data:

In [11]:
# Create a new df with cols we need from raw_homes_df:
# list(raw_homes_df.columns)
homes_df = raw_homes_df[['MunicipalCodeFIPS','StateCodeFIPS','State','RegionID', 'SizeRank', '2000-01-31','2021-12-31','2022-12-31']].copy()

# Clean up column names:
homes_df = homes_df.rename(columns={'MunicipalCodeFIPS': 'county_FIPS', 'StateCodeFIPS': 'state_FIPS', 'RegionName': 'county_name', 'State':'state_abbr','2000-01-31':'home_values_Jan_2000','2021-12-31':'home_values_Dec_2021','2022-12-31':'home_values_Dec_2022'})
homes_df.head() 


,county_FIPS,state_FIPS,state_abbr,RegionID,SizeRank,home_values_Jan_2000,home_values_Dec_2021,home_values_Dec_2022
0,37,6,CA,3101,0,217219.0,821138.0,845596.0
1,31,17,IL,139,1,173902.0,294321.0,312317.0
2,201,48,TX,1090,2,115286.0,258440.0,289591.0
3,13,4,AZ,2402,3,142739.0,433465.0,451699.0
4,73,6,CA,2841,4,225274.0,824324.0,874095.0


In [17]:
# Add the change in home value index from one year prior to newest data, formula from the Zillow site: 
# zhvi_yr_growth = 100 * [ ZHVI_{this month current year} – ZHVI_{this month last year} ] / [ ZHVI_{this month last year} ]
# Create the Zillow Home Value Index growth for one year from 12/21 to 12/22:

homes_df["zhvi_yr_growth"] = 100 * (homes_df["home_values_Dec_2022"] - homes_df["home_values_Dec_2021"]) / (homes_df["home_values_Dec_2021"]) 

homes_df.describe()["zhvi_yr_growth"]
# homes_df["zhvi_yr_growth"] is the percent annual growth from 12/21 to 12/22, some places saw a home value index decline so the min is negative

count    2784.000000
mean       11.023814
std         4.827320
min       -14.695919
25%         7.790821
50%        10.465205
75%        13.885684
max        44.260661
Name: zhvi_yr_growth, dtype: float64

- Then, the Nat'l Risk Index data:

In [18]:
#  Create a new df with cols we need from raw_risk_df:
# list(raw_risk_df.columns)
risk_df = raw_risk_df[['STCOFIPS','COUNTYFIPS','COUNTY','STATE','STATEFIPS','POPULATION','BUILDVALUE','AGRIVALUE','AREA','RISK_SCORE','RISK_RATNG','EAL_SCORE','EAL_RATNG','SOVI_SCORE','SOVI_RATNG','RESL_SCORE','RESL_RATNG','CFLD_RISKS','CFLD_RISKR','DRGT_RISKS',
'DRGT_RISKR','HWAV_RISKS','HWAV_RISKR','HRCN_RISKS',
'HRCN_RISKR','LTNG_RISKS','LTNG_RISKR','RFLD_RISKS',
'RFLD_RISKR','TRND_RISKS','TRND_RISKR','WFIR_RISKS',
 'WFIR_RISKR','WNTW_RISKS','WNTW_RISKR','SHAPE_Length','SHAPE_Area']].copy()

# Clean up risk_df column names:
risk_df = risk_df.rename(columns={'STCOFIPS':'state_county_FIPS','COUNTYFIPS':'county_FIPS', 'COUNTY': 'county_name','STATE':'state','STATEFIPS':'state_FIPS','RISK_SCORE':'risk_index_score','RISK_RATNG':'risk_index_rating','EAL_SCORE':'exp_annual_loss_score','EAL_RATNG':'exp_annual_loss_rating','SOVI_SCORE':'soc_vul_score','SOVI_RATNG':'soc_vul_rating','RESL_SCORE':'comm_res_score','RESL_RATNG':'comm_res_rating','CFLD_RISKS':'coastal_flooding_score','CFLD_RISKR':'coastal_flooding_rating','DRGT_RISKS':'drought_score','DRGT_RISKR':'drought_rating','HWAV_RISKS':'heatwave_score','HWAV_RISKR':'heatwave_rating','HRCN_RISKS':'hurricane_score','HRCN_RISKR':'hurricane_rating','LTNG_RISKS':'lightning_score','LTNG_RISKR':'lightning_rating','RFLD_RISKS':'river_flooding_score','RFLD_RISKR':'river_flooding_rating','TRND_RISKS':'tornado_scores','TRND_RISKR':'tornado_rating','WFIR_RISKS':'wildfire_scores','WFIR_RISKR':'wildfire_rating','WNTW_RISKS':'winterweather_score','WNTW_RISKR':'winterweather_rating'})
risk_df.head() 


,state_county_FIPS,county_FIPS,county_name,state,state_FIPS,POPULATION,BUILDVALUE,AGRIVALUE,AREA,risk_index_score,...,river_flooding_score,river_flooding_rating,tornado_scores,tornado_rating,wildfire_scores,wildfire_rating,winterweather_score,winterweather_rating,SHAPE_Length,SHAPE_Area
0,21115,115,Johnson,Kentucky,21,23356,1924008000,706000,261.958144,9.281419,...,14.575572,Relatively Moderate,9.136885,Relatively Low,13.278676,Relatively Low,14.131237,Relatively Low,1.553484,0.070028
1,21117,117,Kenton,Kentucky,21,159720,18773375000,5390000,160.213975,10.449057,...,8.279166,Relatively Low,24.280149,Relatively Moderate,0.000000,No Rating,12.470040,Relatively Low,1.094178,0.044207
2,21119,119,Knott,Kentucky,21,16346,1170376000,408000,351.517978,10.068395,...,8.755275,Relatively Low,10.174559,Relatively Low,13.363295,Relatively Low,14.466270,Relatively Low,1.694376,0.092941
3,21121,121,Knox,Kentucky,21,31883,2135773000,2534000,386.298435,11.858245,...,14.443835,Relatively Moderate,19.273345,Relatively Moderate,20.352643,Relatively Moderate,19.585915,Relatively Moderate,1.920543,0.101520
4,21123,123,Larue,Kentucky,21,14193,1221343000,41112000,261.539564,4.610900,...,4.055177,Very Low,9.216597,Relatively Low,0.226191,Very Low,7.715952,Very Low,1.830220,0.069616


### Merge Zillow Home df and Nat'l Risk Index df:

In [19]:
# Merge homes_df and risk_df using a left join on the county_FIPS id var and call the merged df the home valu risk df or "hv_risk_df"- this is all county level data:
hv_risk_df = pd.merge(homes_df, risk_df, on = ["state_FIPS","county_FIPS"], how = "left")
hv_risk_df

,county_FIPS,state_FIPS,state_abbr,RegionID,SizeRank,home_values_Jan_2000,home_values_Dec_2021,home_values_Dec_2022,zhvi_yr_growth,state_county_FIPS,...,river_flooding_score,river_flooding_rating,tornado_scores,tornado_rating,wildfire_scores,wildfire_rating,winterweather_score,winterweather_rating,SHAPE_Length,SHAPE_Area
0,37,6,CA,3101,0,217219.0,821138.0,845596.0,2.978549,6037,...,20.446323,Relatively High,37.059979,Relatively High,89.591348,Very High,14.518174,Relatively Low,8.493046,1.036605
1,31,17,IL,139,1,173902.0,294321.0,312317.0,6.114412,17031,...,40.855248,Very High,91.889401,Very High,1.997877,Very Low,56.417423,Very High,3.186134,0.268912
2,201,48,TX,1090,2,115286.0,258440.0,289591.0,12.053475,48201,...,100.000000,Very High,100.000000,Very High,11.810344,Relatively Low,65.334621,Very High,3.879945,0.429804
3,13,4,AZ,2402,3,142739.0,433465.0,451699.0,4.206568,4013,...,20.728803,Relatively High,19.696050,Relatively Moderate,41.924991,Relatively High,6.538783,Very Low,8.119691,2.314110
4,73,6,CA,2841,4,225274.0,824324.0,874095.0,6.037796,6073,...,18.929178,Relatively High,19.203448,Relatively Moderate,83.830573,Very High,6.900459,Very Low,6.453682,1.059064
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2839,3,6,CA,828,3104,NaN,518820.0,550065.0,6.022320,6003,...,0.030130,Very Low,0.844738,Very Low,10.120607,Relatively Low,17.890629,Relatively Moderate,2.174214,0.199044
2840,53,8,CO,2345,3113,NaN,423936.0,459053.0,8.283562,8053,...,3.489408,Very Low,3.452672,Very Low,7.388367,Relatively Low,9.218594,Relatively Low,2.846950,0.297671
2841,79,8,CO,1933,3114,NaN,376561.0,393469.0,4.490109,8079,...,5.984310,Relatively Low,3.483532,Very Low,6.729472,Relatively Low,5.005185,Very Low,2.045067,0.232169
2842,9,49,UT,1648,3129,NaN,301428.0,324937.0,7.799209,49009,...,0.123687,Very Low,1.819058,Very Low,5.382947,Very Low,8.572360,Very Low,2.895317,0.199419


- Save Merged df as .csv:

In [20]:
# Save merged df, hv_risk_df, to .csv in Output Folder and this is ready to manipulate further and save to mongoDB:
hv_risk_df.to_csv("output/hv_risk_df.csv", index=False)

# Read in Data After cleaning/merge:

In [21]:
# Read in happiness data from csv:
hv_risk_file = "output/hv_risk_df.csv"
hv_risk_df = pd.read_csv(hv_risk_file)
hv_risk_df.head() 

,county_FIPS,state_FIPS,state_abbr,RegionID,SizeRank,home_values_Jan_2000,home_values_Dec_2021,home_values_Dec_2022,zhvi_yr_growth,state_county_FIPS,...,river_flooding_score,river_flooding_rating,tornado_scores,tornado_rating,wildfire_scores,wildfire_rating,winterweather_score,winterweather_rating,SHAPE_Length,SHAPE_Area
0,37,6,CA,3101,0,217219.0,821138.0,845596.0,2.978549,6037,...,20.446323,Relatively High,37.059979,Relatively High,89.591348,Very High,14.518174,Relatively Low,8.493046,1.036605
1,31,17,IL,139,1,173902.0,294321.0,312317.0,6.114412,17031,...,40.855248,Very High,91.889401,Very High,1.997877,Very Low,56.417423,Very High,3.186134,0.268912
2,201,48,TX,1090,2,115286.0,258440.0,289591.0,12.053475,48201,...,100.000000,Very High,100.000000,Very High,11.810344,Relatively Low,65.334621,Very High,3.879945,0.429804
3,13,4,AZ,2402,3,142739.0,433465.0,451699.0,4.206568,4013,...,20.728803,Relatively High,19.696050,Relatively Moderate,41.924991,Relatively High,6.538783,Very Low,8.119691,2.314110
4,73,6,CA,2841,4,225274.0,824324.0,874095.0,6.037796,6073,...,18.929178,Relatively High,19.203448,Relatively Moderate,83.830573,Very High,6.900459,Very Low,6.453682,1.059064


In [22]:
hv_risk_df.describe()

,county_FIPS,state_FIPS,RegionID,SizeRank,home_values_Jan_2000,home_values_Dec_2021,home_values_Dec_2022,zhvi_yr_growth,state_county_FIPS,POPULATION,...,drought_score,heatwave_score,hurricane_score,lightning_score,river_flooding_score,tornado_scores,wildfire_scores,winterweather_score,SHAPE_Length,SHAPE_Area
count,2844.000000,2844.000000,2844.000000,2844.000000,1036.000000,2.788000e+03,2.840000e+03,2784.000000,2844.000000,2.844000e+03,...,2844.000000,2844.000000,2130.000000,2833.000000,2844.000000,2844.000000,2833.000000,2844.000000,2844.000000,2844.000000
mean,103.142053,30.079817,1677.743671,1439.927567,114756.305019,2.256325e+05,2.493933e+05,11.023814,30182.959212,1.079753e+05,...,9.237772,7.592806,6.263109,12.694223,9.556998,15.809881,6.374388,13.677201,2.663493,0.263941
std,108.850833,15.249143,931.500432,844.690947,63847.204619,1.566989e+05,1.681956e+05,4.827320,15268.397059,3.274150e+05,...,9.434192,7.826251,7.716075,8.055121,6.876586,9.378186,8.666338,8.642994,2.761869,0.442150
min,1.000000,1.000000,66.000000,0.000000,28879.000000,3.548300e+04,3.984000e+04,-14.695919,1001.000000,6.990000e+02,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.005204,0.000000,0.000000,0.131432,0.000550
25%,35.000000,18.000000,874.750000,710.750000,71779.000000,1.324540e+05,1.463020e+05,7.790821,18100.500000,1.434575e+04,...,2.496065,0.000000,2.728209,7.828624,5.308793,9.644649,1.465168,8.330129,1.642371,0.114593
50%,77.000000,29.000000,1673.500000,1426.500000,101169.000000,1.828940e+05,2.030405e+05,10.465205,29080.000000,3.000500e+04,...,7.422007,6.558139,3.926041,11.017877,8.209145,14.354040,3.184350,12.055682,2.017602,0.161841
75%,133.000000,45.000000,2487.500000,2154.250000,140001.750000,2.718352e+05,3.023508e+05,13.885684,45041.500000,7.623475e+04,...,13.040124,11.136209,6.068763,15.230375,11.996568,20.269387,7.831968,17.481956,2.706163,0.236664
max,840.000000,56.000000,3291.000000,3132.000000,728170.000000,2.224184e+06,2.332809e+06,44.260661,56045.000000,9.818605e+06,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,66.801075,11.197913


_______________________________________________
# Setup for the MongoDB: 

In [2]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [3]:
# Define database for our project called "home_risk_db" and two collections, one for the geoJSON county data ("us-county-boundaries.geojson") named county_bounds and one for the merged .csv file containing the home value/risk index data called "hv_risk":
# db 
db = client.home_risk_db
county_bounds_collection = db.county_bounds
hv_risk_collection = db.hv_risk

In [36]:
# Home value/Risk data:
# Already added, don't run unless the collection is cleared first--
# Insert the hv_risk_df into the hv_risk_collection:
# hv_risk_collection.insert_many(hv_risk_df.to_dict('records'))

# clears collections:
# hv_risk_collection.drop()
# county_bounds_collection.drop()


In [70]:
# Run the following code to fix the encoding in the json file
us_county_path = "resources/us_county_bounds.json"

cur_json = json.load(open(us_county_path, encoding='ISO-8859-1'))
path,ext = os.path.splitext(us_county_path)
new_path =path+"_new"+ext
# Creates a new json file with the encoding set to 'utf-8':
with open(new_path,"w", encoding='utf-8') as jsonfile:
        json.dump(cur_json,jsonfile,ensure_ascii=False)
# us_county = gpd.read_file(new_path, driver='GeoJSON')
# Works to get a geodatafram:


In [75]:
# county boundaries geoJSON data:

# Read in the geoJSON:
with open('resources/us_county_bounds_new.json', 'r') as file:
    cnty_bound_data = json.loads(file.read())

# create 2dsphere index and initialize unordered bulk insert
county_bounds_collection.create_index([("geometry", GEOSPHERE)])
bulk = county_bounds_collection.initialize_unordered_bulk_op()
# For loop to read in the data by each feature:
for feature in cnty_bound_data['features']:
    # append to bulk insert list
    bulk.insert(feature)

# execute bulk operation to the DB
try:
  result = bulk.execute()
  print ("Number of Features successully inserted:", result["nInserted"])
except BulkWriteError as bwe:
  nInserted = bwe.details["nInserted"]
  errMsg = bwe.details["writeErrors"]
  print ("Errors encountered inserting features")
  print ("Number of Features successully inserted:", nInserted)
  print ("The following errors were found:")
  for item in errMsg:
    print ("Index of feature:", item["index"])
    print ("Error code:", item["code"])
    print ("Message (truncated due to data length):", item["errmsg"][0:120], "...")
# Successful with no errors:

/opt/anaconda3/envs/PythonDataOne/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: initialize_unordered_bulk_op is deprecated
  if __name__ == '__main__':


Number of Features successully inserted: 3221


In [ ]:
# Display items in MongoDB collection
# Tested both datasets in the db:
# listings = db.county_bounds.find()

# for listing in listings:
#     print(listing)

In [78]:
# Close the mongo connection:
client.close()

In [40]:
# Function to encode mongoDB object_id:
class MongoJSONEncoder(json.JSONEncoder):
    def default(self, o: Any) -> Any:
        if isinstance(o, ObjectId):
            return str(o)
        if isinstance(o, datetime):
            return str(o)
        return json.JSONEncoder.default(self, o)

In [41]:
homes = hv_risk_collection.find()
# Convert object_id from the homes mongo cursor:
homes_json = MongoJSONEncoder().encode(list(homes))
# Convert to python obj:
homes_obj = json.loads(homes_json)
homes_obj

[{'_id': '63cff9e60c9738d321664a53',
  'county_FIPS': 37,
  'state_FIPS': 6,
  'state_abbr': 'CA',
  'RegionID': 3101,
  'SizeRank': 0,
  'home_values_Jan_2000': 217219.0,
  'home_values_Dec_2021': 821138.0,
  'home_values_Dec_2022': 845596.0,
  'zhvi_yr_growth': 2.9785492816067456,
  'state_county_FIPS': 6037,
  'county_name': 'Los Angeles',
  'state': 'California',
  'POPULATION': 9818605,
  'BUILDVALUE': 950975224000,
  'AGRIVALUE': 154608000,
  'AREA': 4058.07675187911,
  'risk_index_score': 100.0,
  'risk_index_rating': 'Very High',
  'exp_annual_loss_score': 100.0,
  'exp_annual_loss_rating': 'Very High',
  'soc_vul_score': 44.895545920378396,
  'soc_vul_rating': 'Relatively High',
  'comm_res_score': 51.8922,
  'comm_res_rating': 'Relatively Low',
  'coastal_flooding_score': 2.94309868147344,
  'coastal_flooding_rating': 'Very Low',
  'drought_score': 0.485589394679696,
  'drought_rating': 'Very Low',
  'heatwave_score': 17.973939911167,
  'heatwave_rating': 'Relatively High',
 